Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.7.beta8, Release Date: 2022-08-07.
Using Python 3.10.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

## Quotient by $(12)(34)$ in the HC-model

In [1]:
# Let's first demonstrate how we found the quotient curve

# Let's define the curve wrt the invariants of the action (X,Y,Z) -> (X,Z,Y)
# which we call T=Y+Z and V=Y*Z.
Q3.<X,T,V> = QQ[]
# Some algebra is required to calculate this first term
F = X*(T^5-5*T^3*V+5*T*V^2)+X^2*V^2-X^4*V-2*V^3

# We make a check to see that we have done this correctly. 
var('Y Z')
FB = X*(Y^5+Z^5)+(X*Y*Z)^2-X^4*Y*Z-2*(Y*Z)^3
print("Curve is correctly defined in terms of invariants: ",bool(F.subs({T:Y+Z, V:Y*Z})==FB))
print()

# We put this in Weierstrass form using Maple, and the fact we know the curve is genus-2
# by a Riemann Hurwitz argument
poly = QQ['X','V'](F(X,1,V))
MP = Maple()
MP.with_package('algcurves')
v = MP('Weierstrassform({}, X, V, A, B)'.format(poly))
poly_wp = v.op(1)
print("Weierstrassform output:")
print(v)

Curve is correctly defined in terms of invariants:  True

Weierstrassform output:
[-A^6-4*A^5-10*A^3+B^2-4*A-1, (-1+V-X)/(-X^2+V), (8*X^6-5*V*X^4-4*X^5+30*V^2*X^2-65*V*X^3-30*X^4-40*V^2*X+110*V*X^2+65*X^3-10*V*X-55*X^2-14*X+2)/X/(8*X^5-20*X^4+10*X^3+5*X^2-5*X+1), 1/2*(-A^3-2*A^2+2*A-B-1)/(-2*A^3+A^2-A), (1/2*A^4-5/2*A^3-3/2*A-1/2+(1/2*A+1/2)*B)/(4*A^4-4*A^3+5*A^2-2*A+1)]


In [2]:
# Let's verify our claims about X0(50)
# This cell just follows the process in "Defining equations of modular curves $X_0(N)$" (Shimura)
N = 50 
M = ModularFormsRing(N)
S2 = M.modular_forms_of_weight(2).cuspidal_submodule()
g = len(S2.basis()) # = genus = 2
P = 3*g+3
g2, g1 = S2.q_expansion_basis(prec=P)

if not g1[1]==0 and g1[2]==1 and g2[0]==0 and g2[1]==0:
    g1, g2 = (g2, g1)

x = g2/g1
q = x.parent(x.variable())
y = q*x.derivative()/g1

_.<A, B> = QQ[]

d = 2*g+2
F = B^2 - A^d

for k in range(d):
    trunc = (F(x,y).truncate(-d+k+2)*q^(d-k-1)).coefficients()
    if trunc:
        ak = trunc[0]
    else:
        ak = 0
    F -= ak*A^(d-k-1)
print(F)

-A^6 + 4*A^5 + 10*A^3 + B^2 + 4*A - 1


In [3]:
# There will be a more convenient form of this hyperelliptic curve that we find now
var('B2 A2')
QQxy.<A,B> = PolynomialRing(QQ)
poly = QQxy(poly_wp)
poly2 = poly.subs({A: (2+A2)/(2-A2), B:4*B2/(2-A2)^3}).numerator()/16
print("The hyperelliptic curve in terms of B2, A2:")
print(poly2,"\n")

The hyperelliptic curve in terms of B2, A2:
A2^6 - 5*A2^4 - 40*A2^2 + B2^2 - 80 



In [4]:
# From here we read off the two quotients (as per Cassels Flynn) and then find the j-invariant
# and the Cremona label. 
R.<x,y,z> = QQ[]

f = y^2*z + (x^3  - 5*x^2*z - 40*x*z^2 - 80*z^3)
E = Jacobian(f)
print("f: 0 =", f)
print("j =", E.j_invariant().factor())
print("label:", E.label())

print()
f = y^2*z + (z^3  - 5*z^2*x - 40*z*x^2 - 80*x^3)
E = Jacobian(f)
print("f: 0 =", f)
print("j =", E.j_invariant().factor())
print("label:", E.label())

f: 0 = x^3 - 5*x^2*z + y^2*z - 40*x*z^2 - 80*z^3
j = -1 * 2^-5 * 5 * 29^3
label: 50b1

f: 0 = -80*x^3 - 40*x^2*z + y^2*z - 5*x*z^2 + z^3
j = -1 * 2^-1 * 5^2
label: 50a1


## Quotient by $(12)(34)$ in the $\mathbb{P}^4$-model

In [5]:
# Let's now confirm the same results coming via the P4-model
# Let's do something strange, let's define our semi invariants for the order-4 action in advance
Qj.<j> = QQ.extension(polygen(QQ)^2+1)
P3s.<s1, s2, s3, s4> = Qj[]

v2s = matrix([[1,1,1,1],[1,-1, j,-j],[1,1,-1,-1],[1,-1,-j,j]])

s2v = v2s.inverse()
v1, v2, v3, v4 = s2v*vector([s1, s2, s3, s4])

# We then define our two constraints (that is, we have already defined v5 := -v1-v2-v3-v4)
Q2 = v1^2+v2^2+v3^2+v4^2+(v1+v2+v3+v4)^2
Q3 = v1^3+v2^3+v3^3+v4^3-(v1+v2+v3+v4)^3
#Q2.resultant(Q3)

print("The defining equations in terms of the semi-invariants:")
print("Q2: ",Q2)
print("Q3: ",Q3)
print()

# For the 2,2 we introduce s=s1, t=s2*s4, u=s3, v=s2^2
# This allows us to rewrite Q2 and Q3 in terms of our invariants. 
# Note that (remarkably) Q2 and Q3 are polynomials over QQ. 
P3si.<s,t,u,v> = QQ[]
Q2 = 5/4*s^2 + 1/4*u^2 + 1/2*t
Q3 = -15/16*s^3 + 3/16*u*v + 3/16*s*u^2 + 3/8*s*t + 3/16*t^2*u/v

# In writing these, we notice we can eliminate t from Q3 using Q2,
# to get a single polynomial in s,u,v. 
tsub = -2*(5/4*s^2 + 1/4*u^2)
poly = Q3(s,tsub,u,v)
# We extract the polynomial via the numerator 
P3sir.<s,u,v> = QQ[]
poly = P3sir(poly(s,0,u,v).numerator())
print("The defining equation in terms of the invariants:")
print("reduced polynomial: ", poly,"\n")

# We can now use Maple to find a Weierstrass form of this curve, that 
# we know to be elliptic from a Riemann-Hurwitz argument. 
# Take the variables of the reduced polynomial to be u,v, leaving s as a scale,
# and take the coordinates on the new elliptic curve to be x, y. 
wf_data = MP('Weierstrassform({}, u, v, x, y)'.format(poly))
poly_wp = wf_data.op(1)
print("Output of Weierstrassform: ")
print(wf_data, "\n")


# We now do the same check of the j-invariants as before. 
R.<x,y,z> = QQ[]

f = y^2*z + (x^3  + 10*x^2*z + 25*x*z^2 - 100*z^3)
E = Jacobian(f)
print("f: 0 =", f)
print("j =", E.j_invariant().factor())
print("label:", E.label())

print()
f = y^2*z + (z^3  + 10*z^2*x + 25*z*x^2 - 100*x^3)
E = Jacobian(f)
print("f: 0 =", f)
print("j =", E.j_invariant().factor())
print("label:", E.label())

The defining equations in terms of the semi-invariants:
Q2:  5/4*s1^2 + 1/4*s3^2 + 1/2*s2*s4
Q3:  -15/16*s1^3 + 3/16*s2^2*s3 + 3/16*s1*s3^2 + 3/8*s1*s2*s4 + 3/16*s3*s4^2

The defining equation in terms of the invariants:
reduced polynomial:  75/64*s^4*u + 15/32*s^2*u^3 + 3/64*u^5 - 15/8*s^3*v + 3/16*u*v^2 

Output of Weierstrassform: 
[-100*s^6+25*s^4*x^2+10*s^2*x^4+x^6+y^2, u, -10*s^3+2*u*v, x, 1/4*(20*s^3+2*y)/x] 

f: 0 = x^3 + 10*x^2*z + y^2*z + 25*x*z^2 - 100*z^3
j = -1 * 2^-1 * 5^2
label: 400c1

f: 0 = -100*x^3 + 25*x^2*z + y^2*z + 10*x*z^2 + z^3
j = -1 * 2^-5 * 5 * 29^3
label: 400c3


In [6]:
# We check whether the invariants we have are indeed defined over QQ.
P3s.<v1, v2, v3, v4> = Qj[]
s1, s2, s3, s4 = v2s*vector([v1, v2, v3, v4])
s, t, u, v = s1, s2*s4, s3, s2^2
print(s)
print(t)
print(u)
print(v)

v1 + v2 + v3 + v4
v1^2 - 2*v1*v2 + v2^2 + v3^2 - 2*v3*v4 + v4^2
v1 + v2 - v3 - v4
v1^2 - 2*v1*v2 + v2^2 + (2*j)*v1*v3 + (-2*j)*v2*v3 - v3^2 + (-2*j)*v1*v4 + (2*j)*v2*v4 + 2*v3*v4 - v4^2


## Quotient by $(1324)$ in the $\mathbb{P}^4$-model

In [7]:
# To do the quotient by (1324) we now need only follow the process above, but take invariants
# U = u*v = s3*s2^2, V = v^2 = s2^4
P3si.<s,t,U,V> = QQ[]
Q2 = 5/4*s^2 + 1/4*U^2/V + 1/2*t
Q3 = -15/16*s^3 + 3/16*U + 3/16*s*U^2/V + 3/8*s*t + 3/16*t^2*U/V

tsub = -2*(5/4*s^2 + 1/4*U^2/V)
poly = Q3(s,tsub,U,V)
# We extract the polynomial via the numerator 
P3sir.<s,U,V> = QQ[]
poly = P3sir(poly(s,0,U,V).numerator())
print("The defining equation in terms of the invariants:")
print("reduced polynomial: ", poly,"\n")

# We can now use Maple to find a Weierstrass form of this curve, that 
# we know to be elliptic from a Riemann-Hurwitz argument. 
# Take the variables of the reduced polynomial to be u,v, leaving s as a scale,
# and take the coordinates on the new elliptic curve to be x, y. 
wf_data = MP('Weierstrassform({}, U, V, x, y)'.format(poly))
poly_wp = wf_data.op(1)
print("Output of Weierstrassform: ")
print(poly_wp)

The defining equation in terms of the invariants:
reduced polynomial:  75/64*s^4*U*V^2 + 15/32*s^2*U^3*V - 15/8*s^3*V^3 + 3/64*U^5 + 3/16*U*V^3 

Output of Weierstrassform: 
x^3-25/3*s^4*x-2950/27*s^6+y^2


In [8]:
# We now do the same check of the j-invariants as before. 
R.<x,y,z> = QQ[]

f = y^2*z + (x^3  - 25*x*z^2/3 - 2950*z^3/27)
E = Jacobian(f)
print("f: 0 =", f)
print("j =", E.j_invariant().factor())
print("label:", E.label())

f: 0 = x^3 + y^2*z - 25/3*x*z^2 - 2950/27*z^3
j = -1 * 2^-1 * 5^2
label: 400c1


## Quotient by $A_4$

In [12]:
# We utilise the same method as with the birational transform to find the relations amongst
# the symmetric polynomials.
R.<y0, y1, y2, y3, y4, x1, x2, x3, x4> = PolynomialRing(QQ, order='invlex')
II = R.ideal([x1^2 + x2^2 + x3^2 + x4^2 + (x1 + x2 + x3 + x4)^2, 
              x1^3 + x2^3 + x3^3 + x4^3 - (x1 + x2 + x3 + x4)^3])
Q = R.quotient(II)
y0, y1, y2, y3, y4, x1, x2, x3, x4 = Q.gens()

xs = [x1, x2, x3, x4]
IGs = [sum([xi^k for xi in xs]) for k in range(1,5)] + [prod([xs[i]-xs[j] for i in range(4) for j in range(i)])]
I1, I2, I3, I4, I0 = IGs

JJ = Q.ideal(y0-I0, y1-I1, y2-I2, y3-I3, y4-I4)
B = JJ.groebner_basis()

supset = set([y0, y1, y2, y3, y4])

for bi in B:
    if supset.issuperset(set(bi.variables())):
        print(bi)
print()
# We now do the same check of the j-invariants as before. 
R.<x,y,z> = QQ[]
f = x^3 - 373/64*z*x^2 + 431/32*z^2*x - 701/64*z^3 + 1/4*y^2*z
E = Jacobian(f)
print("f: 0 =", f)
print("j =", E.j_invariant().factor())
print("label:", E.label())

y4bar^3 - 373/64*y1bar^4*y4bar^2 + 431/32*y1bar^8*y4bar - 701/64*y1bar^12 + 1/4*y0bar^2
y3bar - y1bar^3
y2bar + y1bar^2

f: 0 = x^3 - 373/64*x^2*z + 1/4*y^2*z + 431/32*x*z^2 - 701/64*z^3
j = 2^-15 * 5 * 211^3
label: 50a4
